# David Ryan's ANTS methond
- 20-25 % increase in price for 15 days
- 20-25 % increase in volume for 15 days
- 12 out of 15 days up

In [ ]:
# The goal is to determine if the average volume over the 15 day range is greater than the 50-day average volume before the range.
# A couple of things to consider. Your range shows 12 days, all calculations are based on using all 15 days in the range.
#Also, the % change is not simply the move from the start to the end of the range, 
#it's calculated as the average change over the range. If you take all 15 closes, 
#get the average and compare to close before the range, the % change is < 20%
# https://www.tradingview.com/script/uEaFv7tm-Ants-Momentum-Volume-and-Price-MVP/